<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex06noteC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex06noteC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


----
## 準備
----

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりできます．
Python は，機械学習・人工知能やデータサイエンスの分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()
import pickle
import torch
from torchvision import models, transforms
from PIL import Image
import json

----
## 1000種類の物体を識別するニューラルネットを動かしてみよう
----






ニューラルネットワークを用いた画像識別の実験をやってみましょう．
いろんなものが識別できた方が楽しいと思いますので，1000種類の物体を識別させてみます．
ただし，そういうことができるニューラルネットワークを一から学習させようとすると大変ですので，ここではすでに学習済みのものを利用することにします．

---
### 準備

#### 準備その1: 実験に使う画像データの準備

**注意: 以下の画像をこの実験以外の目的で使用してはいけません**

In [ ]:
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/animalphoto.pickle
with open('animalphoto.pickle', 'rb') as f:
    hoge = pickle.load(f)

imgList = []
for d in hoge:
    img = Image.frombytes(**d)
    imgList.append(img)
    fig, ax = plt.subplots(1)
    ax.imshow(img)
    ax.axis('off')
    plt.show()

#### 準備その2: クラスの番号とクラスラベルの対応表の作成

In [ ]:
# ImageNet クラスラベルを表すJSONファイルを入手
!wget -nc https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
class_index = json.load(open('imagenet_class_index.json', 'r'))

K = len(class_index)
labels = {}
for ik, key in enumerate(class_index.keys()):
    labels[ik] = f'{class_index[key][0]} {class_index[key][1]}'
    if 276 <= ik < 300: 
        print(ik, labels[ik])

上記の出力は，1000クラスのうちの一部のものの名前を表します．「ネコ」みたいなものも1クラスではなく，281番 'tabby'（ぶち柄の猫），282番 'tiger_cat'（トラ縞の猫）等々のように分かれています．

#### 準備3: 学習済みのニューラルネットワークのパラメータを入手

規模の大きいネットワークでパラメータがたくさんありますので，読み込みに少し時間がかかります．

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16.eval()
print(vgg16)

上記のセルを実行すると，次のことが行われます．
1. VGG16のネットワークモデルを作成する．
1. ImageNetで学習済みのパラメータの値を読み込んでこのネットワークのパラメータに設定する．
1. ネットワークモデルの構造を表示する．

ここで用いている VGG16 というネットワークモデルは，イギリスオックスフォード大学の [Visual Geometry Group(VGG)](https://www.robots.ox.ac.uk/~vgg/) という研究グループが作った VGG-net というニューラルネットワークモデルのうちの，層が16層あるものです．
VGG-net は，2014年に行われた [ILSVRC2014](https://www.image-net.org/challenges/LSVRC/2014/) という画像識別のコンペティションで世界第2位となったニューラルネットワークモデルです．
ILSVRC2014では，[ImageNet](https://www.image-net.org/) という大規模な画像データセットの中から選ばれた 1000 種類の物体の画像約120万枚を学習データとしていました．

画像を扱うニューラルネットでは，**畳み込み層**(convolutional layer)という特別な構造がよく用いられます．
VGG16は，畳み込み層が13層（上記の出力の `Conv2d` というのが畳み込み層），全結合層（`Linear`，階層型ニューラルネットワークとして説明したものはこれ）3層の計16層から成っています（これ以外にもいくつか特殊な層が間に挟まってます）．

参考文献: [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

#### 準備4: 画像の前処理の定義

画像をネットワークに入力できる形式に加工する処理を定義しておきます．


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

---
### 実験1

上記のサンプル画像たちを VGG16 で識別させてみましょう．


In [ ]:
#@title 実験1
#@markdown 以下で 0 から 3 までの整数を選ぶと4つのサンプル画像の中から一つを選ぶことができます．
i = 0 #@param [0, 1, 2, 3] {type: 'raw'}
img = imgList[i]

# 画像を表示
fig, ax = plt.subplots(1)
ax.imshow(img)
ax.axis('off')
plt.show()

# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(imgList[i]), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')

上記の画像の下の出力は，1000個のクラスの中で，ネットワークの出力の値が大きかったもの上位5位までの，出力の値とクラス名を表します．
ネットワークの出力層の活性化関数は softmax ですので，出力の値は0から1でかつ1000子の出力の値すべての和が 1 となっています（一般のロジスティック回帰モデルの説明参照）．
これらの値は，そのクラスと判定することの「確信度」と解釈できます．

---
### 実験2

自分で用意した画像でも実験してみましょう．

- 上記では動物の例ばかりですが，1000のクラスの中にはそれ以外にも様々なものがあります（人間のクラスはありません）
- 面白い／不思議な結果が得られたら takataka に見せてくれると喜びます



(1) まずは，ウェブ等で適当な画像を探して，この Colab notebook を実行しているコンピュータへ保存しましょう．ファイル名が長かったり日本語を含んでいる場合は，短い名前に変更しておくのがよいです．

(2) 以下のセルを実行して，ファイルをアップロードします．

In [ ]:
# Colab へファイルをアップロード
try:
    from google.colab import files
    rv = files.upload()
except:
    print('このコードは Colab 以外の環境では実行できないよ．')

(3) 以下のセルの1行目のファイル名を上記でアップロードしたものに変えて実行しましょう．アルファベット大文字小文字の区別もありますので注意．うまくいけば画像が表示されるはずです．

In [ ]:
myimg = Image.open('hoge.jpg') # ファイル名を変更しましょう
fig, ax = plt.subplots(1)
ax.imshow(myimg)
ax.axis('off')
plt.show()

(4) 以下のセルを実行すれば結果が表示されます．

In [ ]:
# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(myimg), axis=0)
Y = vgg16(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')